In [1]:
import os, getpass
from dotenv import load_dotenv 

# Load environment variables from a .env file
load_dotenv(".env", override=True)

True

In [2]:
def _set_env(var: str):
    # Check if the environment variable is already set
    if not os.environ.get(var):
        # Prompt the user to enter the value for the environment variable
        os.environ[var] = getpass.getpass(f"{var}: ")

# Set the environment variable "GROQ_API_KEY"
_set_env("GROQ_API_KEY")

In [4]:
from pydantic import BaseModel, Field
from typing import Literal
from langchain_core.tools import tool
from langchain_groq import ChatGroq
from langgraph.graph import MessagesState

In [5]:
class WeatherResponse(BaseModel):
    """ Respond to the user with this"""

    temperature: float = Field(description='The temperature in fahrenheit')
    wind_direction: str = Field(description='The direction of the wind in abbreviated form')
    wind_speed: float = Field(description='The Speed of the wind in km/h')

In [ ]:
class WeatherResponse(BaseModel):
    """
    A model representing the weather response.

    Attributes:
        temperature (float): The temperature in Fahrenheit.
        wind_direction (str): The direction of the wind in abbreviated form.
        wind_speed (float): The speed of the wind in km/h.
    """

In [6]:
class AgentState(MessagesState):
    # Final structured response from the agent
    final_response: WeatherResponse

@tool
def get_weather(city: Literal['nyc', 'sf']):
    """ Use this to get weather information"""

    # Return weather information based on the city
    if city == 'nyc':
        return 'It is cloudy in NYC, with 5 mph winds in the North-East direction and a temperature of 70 degrees'
    elif city == 'sf':
        return "It is 75 degrees and sunny in SF, with 3 mph winds in the South-East direction"
    else:
        raise AssertionError("Unknown city")

# List of tools to be used by the model
tools = [get_weather]

# Initialize the ChatGroq model with a specific configuration
model = ChatGroq(model='llama3-8b-8192')

# Bind the tools to the model
model_with_tools = model.bind_tools(tools)

# Configure the model to return structured output
model_with_structured_output = model.with_structured_output(WeatherResponse)

### Single LLM option

## Graph

In [10]:
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode

# List of tools to be used by the model
tools = [get_weather, WeatherResponse]

# Force the model to use tools by passing tool_choice = 'any'
model_with_response_tool = model.bind_tools(tools, tool_choice='any')

# Define the function that calls the model
def call_model(state: AgentState):
    # Invoke the model with the current state messages
    response = model_with_response_tool.invoke(state['messages'])
    # Return the response as a list to be added to the existing messages
    return {'messages': [response]}

# Define the function that responds to the user
def respond(state: AgentState):
    # Construct the final answer from the arguments of the last tool call
    weather_tool_call = state['messages'][-1].tool_calls[0]
    response = WeatherResponse(**weather_tool_call['args'])
  
    tool_message = {
        'type': 'tool',
        'content': 'Here is your structured response',
        'tool_call_id': weather_tool_call['id'],
    }

    # Return the final answer and the tool message
    return {'final_response': response, 'messages': [tool_message]}

# Define the function that determines whether to continue or not
def should_continue(state: AgentState):
    message = state['messages']
    last_message = message[-1]
    # If there is only one tool call and it is the response tool call, respond to the user
    if (
        len(last_message.tool_calls) == 1
        and 
        last_message.tool_calls[0]['name'] == 'WeatherResponse'
    ):
        return 'respond'
    # Otherwise, continue using the tool node
    else:
        return 'continue'
    
# Define a new graph
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model) 
workflow.add_node("respond", respond)
workflow.add_node('tools', ToolNode(tools))

# Set the entry point as 'agent'
# This means that this node is the first one called
workflow.set_entry_point('agent')

# Add conditional edges to determine the flow of the graph
workflow.add_conditional_edges(
    'agent',
    should_continue,
    {
        'continue': 'tools',
        'respond': 'respond'
    }
)

# Add edges to define the transitions between nodes
workflow.add_edge('tools', 'agent')
workflow.add_edge('respond', END)

# Compile the workflow into a graph
graph = workflow.compile()

In [11]:
answer = graph.invoke(input={
                            'messages': [('human',
                                        "what's the weather in SF")]
                            })['final_response']

In [12]:
answer

WeatherResponse(temperature=75.0, wind_direction='S-E', wind_speed=3.0)

### Two LLMs

In [13]:
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage

# Define the function that calls the model
def call_model(state: AgentState):
    # Invoke the model with the current state messages
    response = model_with_tools.invoke(state["messages"])
    # Return the response as a list to be added to the existing messages
    return {"messages": [response]}

# Define the function that responds to the user
def respond(state: AgentState):
    # Call the model with structured output to return a consistent format to the user
    # Convert the last ToolMessage in the conversation to a HumanMessage for the model to use
    response = model_with_structured_output.invoke(
        [HumanMessage(content=state["messages"][-2].content)]
    )
    # Return the final answer
    return {"final_response": response}

# Define the function that determines whether to continue or not
def should_continue(state: AgentState):
    messages = state["messages"]
    last_message = messages[-1]
    # If there is no function call, respond to the user
    if not last_message.tool_calls:
        return "respond"
    # Otherwise, continue using the tool node
    else:
        return "continue"

# Define a new graph
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("respond", respond)
workflow.add_node("tools", ToolNode(tools))

# Set the entry point as 'agent'
workflow.set_entry_point("agent")

# Add conditional edges to determine the flow of the graph
workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "tools",
        "respond": "respond",
    },
)

# Add edges to define the transitions between nodes
workflow.add_edge("tools", "agent")
workflow.add_edge("respond", END)

# Compile the workflow into a graph
graph = workflow.compile()


In [14]:
answer = graph.invoke(input={"messages": [("human", "what's the weather in SF?")]})[
    "final_response"
]

In [15]:
answer

WeatherResponse(temperature=75.0, wind_direction='S-E', wind_speed=3.0)